# Artificial Intelligence
## UEMH3073 / UECS2053 / UECS2153

# Lab 2: Genetic Algorithm

This notebook is an assignment requiring you to investigate the Travelling Salesman Problem. Guidance is provided so you can understand what needs to be done for this assignment as you follow through this lab. Convenience classes and functions/ methods are provided.

You will encounter #TODO in the code cells explaining tasks you need to complete. In other words, you will need to write codes and accomplish the #TODO tasks so that the genetic algorithm functions well and runs correctly. Look for "Replacement starts here" and "Replacement ends here" to know the parts of the codes requiring your inputs.
    

The #TODO tasks and their marks distribution are as follows:
 
a. #TODO1 (10 marks) in the Population Initialization function. You will read a set of cities from the filename when creating an initial population. 

b. #TODO2 (10 marks) in the Parent Selection function. You will replace a dummy parent selection function with Tournament Selection. 

c. #TODO3 (10 marks) in the Parent Selection function. You will replace a dummy parent selection function with Proportional Selection.

d. #TODO4 (10 marks) in the Survival Selection function. You will replace the dummy survival selection function with Merge, Sort & Truncate. 
    
e. #TODO5 (10 marks) in the Crossover function. You will replace the dummy crossover function the Partially Mapped Crossover approach.

f. #TODO6 (10 marks) in the Mutation function. You will replace the dummy mutation function with Insertion Mutation approach. 

g. #TODO7 (10 marks) in Performance Evaluation. You will present performance evaluation for the different Parent Selection functions. 

Marks are also given for: Report Presentation and Formatting (15%) and Code Quality and Comments (15%). More details about this notebook and assignemnt are provided in your lab sheet.

## An Overview of the Travelling Salesman Problem

In the travelling salesman problem, a salesperson wish to find the shortest path that passes through all cities s/he wishes to visit given the coordinates of a set of cities. The salesperson should visit each of the cities once only, and so:

a. Each path consists all cities in the set.

b. Each path visits each of the cities once only. So, none of the cities are visited more than once. 

## Imports

In [2]:
%matplotlib inline

# Please add more imports if you need them 

import random
import time
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint as print 

## Convenience Classes

### City

The City class, which represents a city, possesses the properties of the city and has functions/ methods used for calculating the distance between the city and another city. Each path, represented by a chromosome, is formed by a set of cities.   

In [3]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

### Fitness

The Fitness class, which represents the fitness function, possesses the properties of a path and has functions/methods used for calculating the fitness value of the path, which is based on the distance of the path. 

In [4]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = None
        self.fitness = None
    
    def routeDistance(self):
        if self.distance == None:
            pathDistance = 0.0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i+1 < len(self.route):
                    toCity = self.route[i+1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == None:
        # Fitness function (Simple division) that uses a simple 
        # division that divides one by the distance of the path
            self.fitness = 1 / float(self.routeDistance()) 
            # Note: You must ensure a division by zero does not occur 
        return self.fitness


## Population Initialization  

The population initialization function (or method) performs random initialization. This creates an initial population with completely random chromosomes (or solutions). There are three functions related to population initialization. 

The first function is genCityList() which generates a set of cities from a file.  

In [32]:
def genCityList(filename):
    fileCityList = []
    cityList = []
    
    # TODO 1 (10 marks) - Replace the following codes that generate 10 random cities.
    # Your new implementation must read a set of cities from the filename to be used for creating 
    # an initial population.  
    
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors with slight effects on the fitness value.
    # >0 to <5 marks: Major errors with significant effects on the fitness value. 
    # 0 marks:        No answer is given. 
    
    # -------------- Replacement starts here  ----------------

    # Read the cities from the file
    with open(filename, 'r') as file:  
        next(file)  # Skip the header
        for line in file:
            row = line.strip().split(',')  # Split X and Y coordinates
            fileCityList.append(City(x=int(row[1]),y=int(row[2])))  # 1st column in list is ignored, 2nd and 3rd columns are x and y coordinates
    
    cityList = fileCityList

    
    # --------------- Replacement ends here -----------------
    
    return cityList


The second function is createRoute() which generates a random route (chromosome) from a set of City instances.

In [6]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))  # len is 10, random shuffle the entire list
    return route

The third function is initialPopulation() which calls the second function repeatedly to create an initial population (a list of routes).

In [7]:
def initialPopulation(popSize, cityList):
    population = []
    for i in range(0, popSize):
        population.append(createRoute(cityList))  # Call createRoute to shuffle the list, 3 times seperately
    return population

You can run the above functions using the sample runs below. To do so, simply change the cell type from Markdown to Code.

`Sample run 1` initializes 10 cities in cityList as follows:

In [8]:
cityList = genCityList('cities50.txt') 
print(cityList)

[(48,32),
 (35,46),
 (45,86),
 (57,57),
 (15,34),
 (92,57),
 (100,64),
 (52,35),
 (1,77),
 (75,44)]


`Sample run 2` initializes 10 cities in cityList and creates a population with three routes as follows:

In [9]:
cityList = genCityList('cities50.txt') 
population = initialPopulation(3, cityList)  # 3 different routes, randomly shuffled from the list
print(population)

[[(39,40),
  (52,35),
  (98,61),
  (7,26),
  (35,46),
  (76,88),
  (44,66),
  (84,43),
  (78,60),
  (94,70)],
 [(84,43),
  (94,70),
  (39,40),
  (78,60),
  (44,66),
  (98,61),
  (76,88),
  (7,26),
  (35,46),
  (52,35)],
 [(84,43),
  (98,61),
  (35,46),
  (39,40),
  (76,88),
  (44,66),
  (7,26),
  (94,70),
  (52,35),
  (78,60)]]


## Selection

Parents selection selects chromosomes with high fitness values from a population. Survivor selection selects chromosomes with higher fitness values to form the population of the next generation. The population size is len(population), so we have len(population) in this population. 

### Parent Selection

There are three implementations for parent selection. The first parentSelection() performs **random** selection.

In [10]:
def parentSelection(population, poolSize=None):
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []
    
    for i in range(0, poolSize):
        fitness = Fitness(population[i]).routeFitness()  # Calc. fitness but not use it
        matingPool.append(random.choice(population))
      
    return matingPool

The second parentSelection() performs **Tournament** Selection.

In [11]:
def parentSelection(population, poolSize=None):
    
    # TODO 2 (10 marks) - Replace the dummy parent selection function below with  
    # Tournament Selection.
      
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
    
    # You will need to compare the performance achieved by Random Selection, 
    # Tournament Selection, and Proportional Selection during performance evaluation 
    # later. So, you will run either Random Selection, Tournament Selection, or 
    # Proportional Selection in a simulation run.
    
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []
    
    # ---------------- Replacement starts here ----------------
    
    # Tournament Selection
    for i in range(0, poolSize):    
        tournament = random.sample(population, 2)  # Random pick 2 from population
        best = tournament[0]   # Assume the first one is the best
        for j in range(1, 2):  # Compare the rest with the first one, 2 used because randomly picked 2  
            if Fitness(tournament[j]).routeFitness() > Fitness(best).routeFitness():
                best = tournament[j]
        matingPool.append(best)
    
    # ---------------- Replacement ends here ----------------
    
    return matingPool

The third parentSelection() performs **Proportional** Selection.

In [12]:
def parentSelection(population, poolSize=None):
    
    # TODO 3 (10 marks) - Replace the dummy parent selection function below with  
    # Proportional Selection.
      
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
    
    # You will need to compare the performance achieved by Random Selection, 
    # Tournament Selection, and Proportional Selection during performance evaluation 
    # later. So, you will run either Random Selection, Tournament Selection, or 
    # Proportional Selection in a simulation run.
    
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []
    chromosome_fitness = []
    sumfitness = 0.0
    
    # ---------------- Replacement starts here ----------------
    for i in range(0, poolSize):
        chromosome_fitness.append(Fitness(population[i]).routeFitness())
        sumfitness += Fitness(population[i]).routeFitness()

    # print("Sum fitness = " + str(sumfitness))

    j = 0
    for j in range(0, poolSize):
        choice = random.uniform(0, sumfitness)
        # print("Choice = " + str(choice))
        count = 0

        for k in range(0, poolSize):
            if count < choice:
                count += chromosome_fitness[k]
                # print("Count = " + str(count))
            
                if count >= choice:
                    matingPool.append(population[k])
                    break
    # ---------------- Replacement ends here ----------------
    
    return matingPool

### Survival Selection

In [13]:
def survivorSelection(population, eliteSize):
    
    # TODO 4 (10 marks) - Replace the dummy survival selection function below with  
    # Merge, Sort & Truncate.
      
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
    
    elites = []
    
    # ---------------- Replacement starts here ----------------

    # Sort the population based on fitness score in descending order.
    sortedPopulation = sorted(population, key=lambda x: Fitness(x).routeFitness(), reverse=True)

    # sorted() = a function that returns a new sorted list
    # population = the list to be sorted
    # Fitness (x).routeFitness() =  use Fitness class with x as its input, then calculates the fitness score for the route.
    # reverse = True --> sort the list in descending order (high fitness score is the first in list)

    # Select the top eliteSize routes as elites
    elites = sortedPopulation[:eliteSize]

    # [:eliteSize] = slice the first eliceSize elements
    
    # ---------------- Replacement ends here ----------------
    
    return elites

You can run the above functions using the sample runs below. To do so, simply change the cell type from Markdown to Code. 

`Sample run 1` initializes 10 cities in cityList, creates a population with four routes, and creates a pool of parents as follows:

In [14]:
population = initialPopulation(4, genCityList('cities50.txt'))
print('Initial population') 
print(population)

matingpool = parentSelection(population, 4) 
print('Mating pool')
print(matingpool)

'Initial population'
[[(9,23),
  (95,51),
  (36,1),
  (61,21),
  (44,66),
  (14,74),
  (95,97),
  (33,60),
  (56,77),
  (100,64)],
 [(95,51),
  (44,66),
  (9,23),
  (14,74),
  (61,21),
  (56,77),
  (95,97),
  (100,64),
  (36,1),
  (33,60)],
 [(61,21),
  (9,23),
  (44,66),
  (14,74),
  (33,60),
  (95,51),
  (36,1),
  (56,77),
  (95,97),
  (100,64)],
 [(95,51),
  (14,74),
  (95,97),
  (100,64),
  (44,66),
  (9,23),
  (56,77),
  (61,21),
  (33,60),
  (36,1)]]
'Mating pool'
[[(95,51),
  (14,74),
  (95,97),
  (100,64),
  (44,66),
  (9,23),
  (56,77),
  (61,21),
  (33,60),
  (36,1)],
 [(95,51),
  (14,74),
  (95,97),
  (100,64),
  (44,66),
  (9,23),
  (56,77),
  (61,21),
  (33,60),
  (36,1)],
 [(61,21),
  (9,23),
  (44,66),
  (14,74),
  (33,60),
  (95,51),
  (36,1),
  (56,77),
  (95,97),
  (100,64)],
 [(9,23),
  (95,51),
  (36,1),
  (61,21),
  (44,66),
  (14,74),
  (95,97),
  (33,60),
  (56,77),
  (100,64)]]


`Sample run 2` initializes 10 cities in cityList, creates a population with four routes, select an elite chromosome as follows:

In [15]:
population = initialPopulation(4, genCityList('cities50.txt'))
elites = survivorSelection(population, 1)
print('Initial population')
print(population)
print('Selected elites')
print(elites)

'Initial population'
[[(95,51),
  (57,30),
  (59,3),
  (48,32),
  (76,88),
  (15,34),
  (56,77),
  (7,26),
  (10,31),
  (78,28)],
 [(10,31),
  (95,51),
  (15,34),
  (78,28),
  (7,26),
  (76,88),
  (59,3),
  (57,30),
  (56,77),
  (48,32)],
 [(15,34),
  (95,51),
  (7,26),
  (56,77),
  (76,88),
  (10,31),
  (59,3),
  (78,28),
  (48,32),
  (57,30)],
 [(48,32),
  (7,26),
  (78,28),
  (57,30),
  (15,34),
  (95,51),
  (59,3),
  (56,77),
  (10,31),
  (76,88)]]
'Selected elites'
[[(95,51),
  (57,30),
  (59,3),
  (48,32),
  (76,88),
  (15,34),
  (56,77),
  (7,26),
  (10,31),
  (78,28)]]


## Crossover


Crossover selects two parents, crossover the genetic materials of the parents, and produce one or more children. In the Travelling Salesman Problem, each travelling path must be valid. Each path consists all cities in the set, and each path visits each of the cities once only. So, none of the cities are visited more than once. Exchanging parts of two chromosomes tend to produce invalid paths. As an example, Parent 1 is [2 1 0 7 3 5 4 6] and Parent 2 is [6 1 0 5 2 3 4 7]. One point crossover at midpoint generates Child 1 [2 1 0 7 2 3 4 7] and Child 2 [6 1 0 5 3 5 4 6]. Both children are invalid paths.     

In [38]:
def crossover(parent1, parent2):
    
    # TODO 5 (10 marks) - Replace the dummy crossover function below with 
    # Partially Mapped Crossover approach.
   
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
    
    # ---------------- Replacement starts here ----------------
    size = len(parent1)
    child1 = [None] * size
    child2 = [None] * size

    # Random select two cut points, ensure two are different
    cross_point1 = random.randint(0, size)
    cross_point2 = random.randint(0, size)

    while cross_point1 == cross_point2:
        cross_point2 = random.randint(0, size)
    
    start, end = min(cross_point1, cross_point2), max(cross_point1, cross_point2)

    # Select from parent
    child1[start:end] = parent1[start:end]
    child2[start:end] = parent2[start:end]

    # Create a mapping of the genes in the selected range
    p1_map = {value: index for index, value in enumerate(parent1)}
    p2_map = {value: index for index, value in enumerate(parent2)}

    # Fill the rest of the child with the remaining genes from the other parent
    for i in range(size):
        if i < start or i >= end:
            if child1[i] is None:
                child1[i] = parent2[i]
                child1_index = p2_map[child1[i]]
                if child1_index < start or child1_index >= end:
                    child2[i] = parent1[i]
                else:
                    child2[i] = parent1[child1_index]
            if child2[i] is None:
                child2[i] = parent1[i]
                child2_index = p1_map[child2[i]]
                if child2_index < start or child2_index >= end:
                    child1[i] = parent2[i]
                else:
                    child1[i] = parent2[child2_index]

    # ---------------- Replacement ends here ----------------
    
    return child1, child2

Crossover selects two parents from the mating pool to produce a new generation of the same size.

In [17]:
def breedPopulation(matingpool):
    children = []
    
    # Choosing parents in their order of presence in the mating pool. Choosing parents
    # in a random manner is possible. 
    
    for i in range(1, len(matingpool), 2):
        child1, child2 = crossover(matingpool[i-1], matingpool[i])
        children.append(child1)
        children.append(child2)
    
    return children

You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes 2 chromosomes in the population, and performs crossover among the two parents. 

In [41]:
population = initialPopulation(2, genCityList('cities500.txt'))
parent1, parent2 = population
child1, child2 = crossover(parent1, parent2)
print('Parents')
print(parent1)
print(parent2)
print('Children')
print(child1)
print(child2)

'Parents'
[(82,36),
 (74,70),
 (47,99),
 (51,47),
 (9,100),
 (15,34),
 (77,29),
 (93,27),
 (84,5),
 (94,40),
 (80,100),
 (0,41),
 (92,13),
 (44,64),
 (7,74),
 (76,4),
 (98,49),
 (45,95),
 (57,81),
 (15,8),
 (74,54),
 (39,40),
 (15,6),
 (15,72),
 (66,13),
 (63,59),
 (58,21),
 (37,30),
 (17,89),
 (62,33),
 (89,31),
 (25,43),
 (37,59),
 (96,29),
 (76,63),
 (67,4),
 (61,15),
 (98,26),
 (57,5),
 (38,99),
 (60,50),
 (80,68),
 (7,2),
 (48,35),
 (89,58),
 (39,59),
 (58,83),
 (59,86),
 (29,58),
 (49,34),
 (44,4),
 (44,54),
 (70,93),
 (35,31),
 (66,25),
 (95,71),
 (78,60),
 (68,13),
 (34,83),
 (27,81),
 (12,92),
 (10,69),
 (35,67),
 (60,60),
 (51,32),
 (7,94),
 (75,71),
 (93,49),
 (12,98),
 (45,86),
 (76,54),
 (39,46),
 (37,66),
 (69,58),
 (39,27),
 (81,65),
 (69,44),
 (80,24),
 (48,62),
 (86,45),
 (12,22),
 (14,12),
 (0,29),
 (61,63),
 (96,59),
 (45,71),
 (15,51),
 (13,2),
 (74,98),
 (60,35),
 (45,81),
 (85,53),
 (35,42),
 (31,73),
 (65,45),
 (49,31),
 (89,69),
 (71,56),
 (20,92),
 (57,57),
 (9

## Mutation

Mutation mutates a single chromosome to get a mutated chromosome so that genetic algorithm can converge to a shorter path quickly. In the Travelling Saleman Problem, a mutated chromosome must be a valid path. As an example, the insertion mutation randomly inserts a single gene in the [1 2 3 4 5 6 7 8 9 10] chromosome to generate the [1 2 4 5 6 7 3 8 9 10] mutated chromosome. Step 1: select a gene randomly, Step 2: insert this gene into a randomly selected location.

In [19]:
def mutate(route, mutationProbability):
    
    # TODO 6 (10 marks) - Replace the dummy mutation function below with Insertion Mutation.
    # The dummy mutation function simply swaps a city with the city before it.  
   
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
     
    mutated_route = route[:]
    for i in range(len(route)):
        if (random.random() < mutationProbability):
            # mutationProbability is the probability of a gene undergoing mutation
            
            # ---------------- Replacement starts here ----------------
            # print(i)
            city = mutated_route.pop(i)
            insert_point = random.randint(0, len(mutated_route))
            mutated_route.insert(insert_point, city)

            # print(mutated_route)
            # ---------------- Replacement ends here ----------------
    return mutated_route

Mutation runs over the entire population and mutates each chromosome in the population with a small mutationProbability. 

In [20]:
def mutation(population, mutationProbability):
    mutatedPopulation = []
    for i in range(0, len(population)):
        mutatedIndividual = mutate(population[i], mutationProbability)
        mutatedPopulation.append(mutatedIndividual)
    return mutatedPopulation

You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes a route comprised of 10 cities in cityList, and then mutates it as follows:

In [21]:
route = genCityList('cities50.txt')
mutated = mutate(route, 1)  # Give a pretty high chance for mutation
print('Original route')
print(route)
print('Mutated route')
print(mutated)

'Original route'
[(26,39),
 (10,34),
 (95,97),
 (34,83),
 (70,32),
 (59,3),
 (15,46),
 (56,77),
 (80,68),
 (33,60)]
'Mutated route'
[(70,32),
 (10,34),
 (95,97),
 (26,39),
 (59,3),
 (34,83),
 (80,68),
 (33,60),
 (56,77),
 (15,46)]


## Running One Generation (or Interation)

Here, we run one generation of genetic algorithm. 

In [22]:
def oneGeneration(population, eliteSize, mutationProbability):
    
    # First we preserve the elites
    elites = survivorSelection(population, eliteSize)
    
    # Then we calculate what our mating pool size should be and generate
    # the mating pool
    poolSize = len(population) - eliteSize
    matingpool = parentSelection(population, poolSize)
        
    # Then we perform crossover on the mating pool
    children = breedPopulation(matingpool)
    
    # We combine the elites and children into one population
    new_population = elites + children
    
    # We mutate the population
    mutated_population = mutation(new_population, mutationProbability)
        
    return mutated_population

You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes a population comprised of 5 chromosomes based on 10 cities in cityList, and then run one generation (or iteration) of genetic algorithm as follows:

In [23]:
population = initialPopulation(5, genCityList('cities50.txt'))
eliteSize = 1
mutationProbability = 0.01
new_population = oneGeneration(population, eliteSize, mutationProbability)
print('Initial population')
print(population)
print('New population')
print(new_population)

'Initial population'
[[(35,46),
  (9,23),
  (59,3),
  (70,32),
  (77,16),
  (98,61),
  (7,26),
  (10,34),
  (89,52),
  (76,63)],
 [(77,16),
  (10,34),
  (76,63),
  (9,23),
  (70,32),
  (89,52),
  (59,3),
  (35,46),
  (98,61),
  (7,26)],
 [(9,23),
  (89,52),
  (77,16),
  (70,32),
  (35,46),
  (10,34),
  (7,26),
  (76,63),
  (98,61),
  (59,3)],
 [(9,23),
  (59,3),
  (77,16),
  (76,63),
  (89,52),
  (10,34),
  (98,61),
  (35,46),
  (7,26),
  (70,32)],
 [(89,52),
  (70,32),
  (98,61),
  (59,3),
  (7,26),
  (35,46),
  (76,63),
  (77,16),
  (9,23),
  (10,34)]]
'New population'
[[(35,46),
  (9,23),
  (59,3),
  (70,32),
  (77,16),
  (98,61),
  (7,26),
  (10,34),
  (89,52),
  (76,63)],
 [(35,46),
  (9,23),
  (76,63),
  (9,23),
  (70,32),
  (89,52),
  (59,3),
  (35,46),
  (98,61),
  (59,3)],
 [(77,16),
  (10,34),
  (59,3),
  (70,32),
  (77,16),
  (98,61),
  (7,26),
  (10,34),
  (89,52),
  (76,63)],
 [(9,23),
  (59,3),
  (77,16),
  (77,16),
  (76,63),
  (89,52),
  (10,34),
  (98,61),
  (35,46),
 

## Running Many Generations (or Interations) 

In [40]:
filename = 'cities500.txt'
popSize = 20
eliteSize = 5
mutationProbability = 0.01
iteration_limit = 100

cityList = genCityList(filename)

population = initialPopulation(popSize, cityList)
distances = [Fitness(p).routeDistance() for p in population]
min_dist = min(distances)
print("Best distance for initial population: " + str(min_dist))

for i in range(iteration_limit):
    population = oneGeneration(population, eliteSize, mutationProbability)
    distances = [Fitness(p).routeDistance() for p in population]
    index = np.argmin(distances)
    best_route = population[index]
    min_dist = min(distances)
    print("Best distance for population in iteration " + str(i) +
          ": " + str(min_dist))

print("Optimal path is " + str(best_route)) 

    # TODO 7 (10 marks) - Performance Evaluation. You will present the performance achieved 
    # by different parent selection function. You will compare the 
    # performance achieved by Random Selection, Tournament Selection, and Proportional Selection. 
   
    # Marking scheme: 
    # 7 to 10 marks:  In-depth performance evaluation. Optimal routes are found. 
    # 5 to <7 marks:  Clear understanding of performance evaluation.
    # >0 to <5 marks: Inaccurate or unclear understanding of performance evaluation. 
    # 0 marks:        No answer is given. 
    
  

'Best distance for initial population: 25659.1655413889'
'Best distance for population in iteration 0: 25868.41234222961'
'Best distance for population in iteration 1: 25427.723502232635'
'Best distance for population in iteration 2: 24934.694564615806'
'Best distance for population in iteration 3: 24916.308601024117'
'Best distance for population in iteration 4: 25039.8812247397'
'Best distance for population in iteration 5: 24992.408802940517'
'Best distance for population in iteration 6: 24770.641510876372'
'Best distance for population in iteration 7: 24430.31194490203'
'Best distance for population in iteration 8: 24472.581299428282'
'Best distance for population in iteration 9: 24470.63997648192'
'Best distance for population in iteration 10: 24326.595734823946'
'Best distance for population in iteration 11: 24297.451351190917'
'Best distance for population in iteration 12: 24208.28290435985'
'Best distance for population in iteration 13: 24221.970144769777'
'Best distance for po